This notebook shows how to retreieve alerts belonging to either a topic (permanent) or query (temporary) archive token, and then filter these through matching with a (remote) catalog hosted at DESY.

In [ ]:
import requests, os
from astropy.time import Time

In [ ]:
from ampel.log.AmpelLogger import AmpelLogger
from ampel.contrib.hu.t0.RedshiftCatalogFilter import RedshiftCatalogFilter

from ampel.ztf.t0.load.ZTFArchiveAlertLoader import ZTFArchiveAlertLoader
from ampel.ztf.alert.ZiAlertSupplier import ZiAlertSupplier


In [ ]:
header = {"Authorization": "bearer "+os.environ["ARCHIVE_TOKEN"]}
base_url = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3'

## A. Filter parameters
We use a redshift catalog filter, based on a selection of nearby galaxies from NED.

In [ ]:
# This filter is based on ned and has a lot of parameters which can be changed
RedshiftCatalogFilter??

In [ ]:
# Decent filter parameters
filter_config = {
    "catalog_match_radius": 30,
    "min_z": 0.002,
    "max_z": 0.025,
    "max_tspan": 5.,        # Max total detection age in alert
    "max_archive_tspan": 5.,        # Max total detection age in IPAC DB
    "min_rb": 0.3,             # real bogus score
    "min_drb": 0.995,         # deep learning real bogus score 
    # Should be set during proper install
    "resource": {"ampel-ztf/catalogmatch":"https://ampel.zeuthen.desy.de/api/catalogmatch/"},
}

## B. Getting a handle to alerts.
A _resume token_, which points to a specific set of alerts. We will here directly query the alert archives - other notebooks explore other methods for selecting alerts. 

In [ ]:
delta_t = 4.

In [ ]:
query = {
  "jd": {
    "$gt": Time.now().jd-delta_t,
    "$lt": Time.now().jd
  },
  "candidate": {
    "drb": {
      "$gt": 0.995
    },
    "magpsf": {
      "$gt": 16.
    },
    "ndethist": {
      "$gt": 0,
      "$lte": 5
    },
# Possible select solar system objects (if allowing few ndet)
#    "ssdistnr": {
#      "$gt": 20.,
#    },
#    "ssdistnr": {
#      "$lt": 0.
#    },
    "isdiffpos": {"$in": ["t", "1"]},
  }
}

In [ ]:
endpoint = 'https://ampel.zeuthen.desy.de/api/ztf/archive/v3/streams/from_query?'
header = {"Authorization": "bearer "+os.environ["ARCHIVE_TOKEN"]}

In [ ]:
response = requests.post(endpoint, headers=header, json=query )

In [ ]:
response.ok

In [ ]:
resume_token = response.json()['resume_token']

### 4. Configure and load a filter
As defined above

In [ ]:
logger = AmpelLogger.get_logger()

In [ ]:
t0filter = RedshiftCatalogFilter( **filter_config, logger=logger )
t0filter.post_init()

## C. Iterate through filters from the stream, checking whether the filter would accept them

In [ ]:
config = {'archive':"https://ampel.zeuthen.desy.de/api/ztf/archive/v3", 
          "stream": resume_token}   # From above

In [ ]:
accepted_alerts = []
alertcount = 0

In [ ]:
try:
    alertloader = ZTFArchiveAlertLoader(**config)
    for alert in alertloader.get_alerts():
        alertcount += 1
        filter_accept = t0filter.process( ZiAlertSupplier.shape_alert_dict( alert, [] ) )
        if filter_accept:
            accepted_alerts.append(alert)
            print('... accepted')
        if alertcount % 100 == 0:
            print('Parsed {} alerts, out of which {} passed'.format(alertcount, len(accepted_alerts)))
except requests.exceptions.HTTPError as e:
    status_code = e.response.status_code
    if status_code==423:
        print('HTTP error {}: likely caused by server staging process. Wait and try again.'.format(status_code) )
    else:
        raise e


In [ ]:
print(f'Number of alerts inspected: {alertcount}')

In [ ]:
print('Total number of alertsselected:', len( accepted_alerts ) )

In [ ]:
set( [a['objectId'] for a in accepted_alerts] )